In [76]:
import numpy as np
import pandas as pd
import re

In [77]:
# totaldata = pd.read_csv('payload_all.csv',encoding='utf-8')
# totaldata.dropna(inplace=True)
# totaldata.iloc[6531]

train_data = pd.read_csv('train_data.csv',encoding='utf-8')
test_data = pd.read_csv('test_data.csv',encoding='utf-8')
train_data.columns = ['i', 'payload','type']
train_data.drop(['i'], axis=1,inplace=True)
train_data.dropna(inplace=True)
test_data.columns = ['i', 'payload','type']
test_data.drop(['i'], axis=1,inplace=True)
test_data.dropna(inplace=True)
# print(test_data.isnull().values.any())
train_data.loc[train_data['type'] == 'norm', 'class'] = 0
train_data.loc[train_data['type'] == 'xss', 'class'] = 1
test_data.loc[test_data['type'] == 'norm', 'class'] = 0
test_data.loc[test_data['type'] == 'xss', 'class'] = 1


# test_data.type.unique()

# train_data.head()

In [78]:
def generate_token_sent(s):
#     print('new s incoming')
#     print(s)
    # Convert to lowercases
    try:
        s = s.lower()
    except:
        pass
#         print(s)
    # Replace all non-alphanumeric characters with spaces
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    
    
    # Break sentence in the token, remove empty tokens
    tokens = [token for token in s.split(" ") if token != ""]
    
    #construct new sentence with tokens
    sent = ""
    for i in range(len(tokens)):
        if i == 0:
            sent = sent + tokens[0]
        #elif i != (len(tokens)-1):
        #    sent = sent + " " + tokens[i]
        else:
            sent = sent + " " + tokens[i]#sent = sent + tokens[i]
#     print(sent)
    return sent
    

def generate_token_sent_col(arr):
    new_arr = []
#     print(arr.size)
    for i in range(arr.size):
        new_arr.append(generate_token_sent(arr.iloc[i]))
    new_arr = np.array(new_arr)
    return new_arr


In [79]:
# totaldata.drop(totaldata.index[[6531,6533]],inplace=True)
trainpayload = train_data.payload
trainresult = train_data['class']

testpayload = test_data.payload
testresult = test_data['class']

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(payload, result, test_size=0.33, random_state=42)

# print(tot_payload.iloc[6531])

trainpayload_new = generate_token_sent_col(trainpayload)
testpayload_new = generate_token_sent_col(testpayload)


# train_payload = generate_token_sent_col(X_train)
# test_payload = generate_token_sent_col(X_test)

# print(testpayload_new.shape)

In [80]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
count_vect.fit(trainpayload_new)
train_bow = count_vect.transform(trainpayload_new)
test_bow = count_vect.transform(testpayload_new)

# X_train_counts = count_vect.fit_transform(train_payload)
# X_test_counts = count_vect.fit_transform(test_payload)
print(train_bow.shape)
print(test_bow.shape)
# print(X_test_counts.shape)

# X_train, X_test, y_train, y_test = train_test_split(X_total_counts, result, test_size=0.33, shuffle=True, random_state=42)

(60295, 82670)
(9376, 82670)


In [71]:
from sklearn.svm import SVC
clf = SVC(gamma='auto',kernel='linear')
clf.fit(train_bow, trainresult)
y_pred = clf.predict(test_bow)

from sklearn.metrics import accuracy_score
accuracy_score(testresult, y_pred)

0.987094709897611

In [74]:
from sklearn.metrics import f1_score
f1_score(testresult, y_pred, average='micro')  

0.987094709897611

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(trainpayload_new)
Tfidf_trainpayload = vectorizer.transform(trainpayload_new)
Tfidf_testpayload = vectorizer.transform(testpayload_new)
print(Tfidf_trainpayload.shape)
print(Tfidf_testpayload.shape)

# TfidfX_train, TfidfX_test, Tfidfy_train, Tfidfy_test = train_test_split(Tfidf_payload, result, test_size=0.33, random_state=42)
from sklearn.svm import SVC
clf = SVC(gamma='auto',kernel='linear')
clf.fit(Tfidf_trainpayload, trainresult)
tfidf_pred = clf.predict(Tfidf_testpayload)

from sklearn.metrics import accuracy_score
accuracy_score(testresult, tfidf_pred)
f1_score(testresult, tfidf_pred, average='micro')  

(60295, 82670)
(9376, 82670)


0.9811220136518771

In [ ]:
from nltk import ngrams

def makengrams(sentence,n):
    grams = ngrams(sentence.split(), n)
    for g in grams:
      print g
def ngrampayload(payload):
    for each in payload:
        

In [81]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_bow, trainresult)
y_pred = neigh.predict(test_bow)

from sklearn.metrics import accuracy_score
accuracy_score(testresult, y_pred)

0.3342576791808874